# Forest Data Explore
**Source:** Sassan Satchi from NASA JPL provided annual forest flux data for us to use in TRACE Places.
**Goal:** Look at single year of data to better understand dataset. Fairly messy scratchwork in here.

@mattyarri

**Last Updated:** December 3, 2021

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import statsmodels
import numpy as np
import geopandas as gpd
from keplergl import KeplerGl

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pio.templates.default = "none"
%config InlineBackend.figure_format ='retina'

## Data Read-in + Messy Column Check

In [15]:
import pandas
url = "https://docs.google.com/spreadsheets/d/18jO9Rccs_pWSBVUEGbQWRVOAbddvLr4d/gviz/tq?tqx=out:csv"
df = pandas.read_csv(url)
df['Label'] = df['Country'] + ', ' + df['Province']
df.head()

,Unnamed: 0,Country,Province,Forest Area (ha),Nonforest Area (ha),Forest loss Yr-2019 (ha),Forest C Yr-2019 (GgC),Nonforest C Yr-2019 (GgC),Total C Yr-2019 (GgC),Deforestation Emission Yr-2019 (GgC),Degradation Emission Yr-2019 (GgC),Fire Emission Yr-2019 (GgC),Removal Yr-2019 (tons C),Label
0,16,Afghanistan,Khost,5781.328964,26786.93542,6.954579,0.587734,4644.850254,4645.438194,0.132179,0.000000,0.593043,44975.32547,"Afghanistan, Khost"
1,17,Afghanistan,Kunar,118422.961400,113126.77000,14.028640,1706.720829,12172.557830,13879.279140,0.656149,5.996800,0.000000,-214932.54600,"Afghanistan, Kunar"
2,19,Afghanistan,Laghman,15730.001830,30937.96692,0.659929,13.940511,5662.818432,5676.758289,0.025304,0.139418,0.000000,-69262.23636,"Afghanistan, Laghman"
3,21,Afghanistan,Nangarhar,11270.898440,26970.36133,0.286653,72.677568,9697.941780,9770.619392,0.013591,0.000000,0.000000,174216.98570,"Afghanistan, Nangarhar"
4,23,Afghanistan,Nuristan,53666.259770,84450.43945,0.876654,497.455329,27199.813840,27697.269440,0.043111,0.424497,0.000000,229840.91940,"Afghanistan, Nuristan"


In [4]:
df.sample(10)

,Unnamed: 0,Country,Province,Forest Area (ha),Nonforest Area (ha),Forest loss Yr-2019 (ha),Forest C Yr-2019 (GgC),Nonforest C Yr-2019 (GgC),Total C Yr-2019 (GgC),Deforestation Emission Yr-2019 (GgC),Degradation Emission Yr-2019 (GgC),Fire Emission Yr-2019 (GgC),Removal Yr-2019 (tons C)
658,898,Ghana,Volta,4.596653e+05,1.321548e+06,6756.485086,11469.543460,49239.154820,60708.698270,418.146253,110.987842,1780.133128,-3.045126e+06
507,676,Dominican Republic,Santiago,2.098419e+05,4.757260e+04,808.502594,10539.917950,2800.414324,13340.332030,39.313544,33.849936,0.000000,5.209476e+03
1129,1535,Lesotho,Mokhotlong,5.868011e+02,3.620173e+04,1.178126,0.196975,234.380171,234.577149,0.009999,0.000000,3.664005,1.379594e+05
547,743,Ecuador,Napo,1.063027e+06,6.984848e+04,1919.636177,99059.303280,4073.725224,103133.026100,259.499371,277.521193,0.000000,-1.450686e+06
1985,2776,Tajikistan,Khatlon,6.642281e+03,6.871157e+04,0.662491,0.000000,18104.467390,18104.467390,0.000240,0.000000,0.012292,-5.456039e+05
1895,2657,Syria,Hamah,2.813715e+04,2.282567e+04,704.535824,176.525250,6310.898781,6487.423897,41.677795,0.000000,75.397484,-1.797403e+05
876,1151,Ireland,Roscommon,1.334149e+04,4.465040e+04,414.689639,902.375400,2633.725882,3536.101341,28.482206,0.000000,0.000000,5.363511e+04
1126,1530,Lesotho,Butha-Buthe,1.421505e+02,3.471821e+04,1.057096,0.000000,547.990739,547.990739,0.006007,0.000000,4.121695,-5.559354e+04
2056,2899,Turkey,Ordu,3.892000e+05,8.966919e+04,384.405743,20464.727400,3594.987869,24059.715270,28.380558,0.000000,0.479194,3.330463e+05
1485,2097,Philippines,Cavite,4.765931e+04,2.232227e+04,97.412782,2645.177841,1848.346829,4493.525028,6.702966,11.742316,0.000000,-2.151659e+04


In [6]:
# Let's check real quick that the columns add in the way we think they do
df[['Forest C Yr-2019 (GgC)','Nonforest C Yr-2019 (GgC)','Deforestation Emission Yr-2019 (GgC)',
    'Degradation Emission Yr-2019 (GgC)','Fire Emission Yr-2019 (GgC)','Removal Yr-2019 (tons C)',
    'Total C Yr-2019 (GgC)']]

,Forest C Yr-2019 (GgC),Nonforest C Yr-2019 (GgC),Deforestation Emission Yr-2019 (GgC),Degradation Emission Yr-2019 (GgC),Fire Emission Yr-2019 (GgC),Removal Yr-2019 (tons C),Total C Yr-2019 (GgC)
0,0.587734,4644.850254,0.132179,0.000000,0.593043,4.497533e+04,4645.438194
1,1706.720829,12172.557830,0.656149,5.996800,0.000000,-2.149325e+05,13879.279140
2,13.940511,5662.818432,0.025304,0.139418,0.000000,-6.926224e+04,5676.758289
3,72.677568,9697.941780,0.013591,0.000000,0.000000,1.742170e+05,9770.619392
4,497.455329,27199.813840,0.043111,0.424497,0.000000,2.298409e+05,27697.269440
...,...,...,...,...,...,...,...
2372,3193.665028,92745.086670,149.902105,15.953777,5682.972431,-2.566144e+06,95938.758850
2373,2083.210468,84284.034730,38.672268,0.346057,119.506553,2.351748e+05,86367.240910
2374,1062.944412,121544.433600,76.554701,2.450490,2247.374058,-1.671693e+06,122607.376100
2375,97.951524,60811.439510,5.060663,0.000000,142.735392,7.221533e+05,60909.393310


In [32]:
component_columns = ['Forest C Yr-2019 (GgC)','Nonforest C Yr-2019 (GgC)','Deforestation Emission Yr-2019 (GgC)',
    'Degradation Emission Yr-2019 (GgC)','Fire Emission Yr-2019 (GgC)']
df['Test Sum'] = df[component_columns].sum(axis=1)
df[['Test Sum','Total C Yr-2019 (GgC)']]

(df['Test Sum'] - df['Total C Yr-2019 (GgC)']).sum() # Check for parity everywhere
px.histogram((df['Test Sum'] - df['Total C Yr-2019 (GgC)']),title = 'Parity Check for Total C')

printmd("**--> Hmm, definitely off a little bit here still**")

,Test Sum,Total C Yr-2019 (GgC)
0,4646.163210,4645.438194
1,13885.931608,13879.279140
2,5676.923665,5676.758289
3,9770.632938,9770.619392
4,27697.736777,27697.269440
...,...,...
2372,101787.580012,95938.758850
2373,86525.770076,86367.240910
2374,124933.757260,122607.376100
2375,61057.187089,60909.393310


5213535.893855852

**--> Hmm, definitely off a little bit here still**

In [25]:
#Let's plot these components real quick just to visually take a look at how they add up
t_df = df[component_columns + ['Total C Yr-2019 (GgC)','Label']].sample(4)
t_df
fig = px.bar(t_df.melt(id_vars = 'Label'),facet_row='Label',x='variable',y='value',color='variable',height = 600)
fig = fig.update_yaxes(matches=None)
fig

,Forest C Yr-2019 (GgC),Nonforest C Yr-2019 (GgC),Deforestation Emission Yr-2019 (GgC),Degradation Emission Yr-2019 (GgC),Fire Emission Yr-2019 (GgC),Total C Yr-2019 (GgC),Label
1407,39403.69415,18566.19072,9.627111,78.001089,759.752929,57969.88297,"Nepal, Far-Western"
625,64496.91010,44455.88303,307.256013,0.000000,0.000000,108952.77400,"France, Provence-Alpes-Côte d'Azur"
45,0.00000,30526.46065,0.019449,0.000000,51.746096,30526.46065,"Argentina, Mendoza"
2262,40054.77524,69240.34882,211.677373,224.162757,937.174618,109295.12790,"Venezuela, Barinas"


In [39]:
#It looks like the "Total C" column is solely a sum of "Forest C" and "Nonforest C"? Let's check that to be sure
df['Test Sum'] = df['Forest C Yr-2019 (GgC)'] + df['Nonforest C Yr-2019 (GgC)']
df[['Test Sum','Total C Yr-2019 (GgC)']]
(df['Test Sum'] - df['Total C Yr-2019 (GgC)']).sum() # Check for parity everywhere
px.histogram((df['Test Sum'] - df['Total C Yr-2019 (GgC)']),title = 'Parity Check for Total C')
printmd('**--> This looks fairly confirmed as just being the sum of Forest and Non-Forest Flux. Error of 1 is essentially zero**')

,Test Sum,Total C Yr-2019 (GgC)
0,4645.437988,4645.438194
1,13879.278659,13879.279140
2,5676.758943,5676.758289
3,9770.619348,9770.619392
4,27697.269169,27697.269440
...,...,...
2372,95938.751698,95938.758850
2373,86367.245198,86367.240910
2374,122607.378012,122607.376100
2375,60909.391034,60909.393310


-1.7853421096266118

**--> This looks fairly confirmed as just being the sum of Forest and Non-Forest Flux. Error of 1 is essentially zero**

In [35]:
#Let's plot that total C column and see if they're all positive values or not
px.histogram(df['Total C Yr-2019 (GgC)'],title = 'Distribution of "Total C" by Subprovince')

#Let's plot all of the subcomponents as well, just to get a sense of sign distributions
for c in component_columns:
    px.histogram(df[c],title = c + ': Subprovince Distribution')
    
#Plot removal as well
px.histogram(df['Removal Yr-2019 (tons C)'], title = 'Removal Yr-2019 (tons C): Subprovince Distribution')

In [40]:
#Triple check: let's make sure carbon removal isn't in the total column?
df['Test Sum'] = df['Forest C Yr-2019 (GgC)'] + df['Nonforest C Yr-2019 (GgC)'] + df['Removal Yr-2019 (tons C)']
df[['Test Sum','Total C Yr-2019 (GgC)']]
(df['Test Sum'] - df['Total C Yr-2019 (GgC)']).sum() # Check for parity everywhere
px.histogram((df['Test Sum'] - df['Total C Yr-2019 (GgC)']),title = 'Parity Check for Total C')
printmd('**--> Definitely not, the carbon removal column adds a lot of noise/error**')

,Test Sum,Total C Yr-2019 (GgC)
0,4.962076e+04,4645.438194
1,-2.010533e+05,13879.279140
2,-6.358548e+04,5676.758289
3,1.839876e+05,9770.619392
4,2.575382e+05,27697.269440
...,...,...
2372,-2.470205e+06,95938.758850
2373,3.215420e+05,86367.240910
2374,-1.549086e+06,122607.376100
2375,7.830627e+05,60909.393310


-3403165801.2485766

**--> Definitely not, the carbon removal column adds a lot of noise/error**